<a href="https://colab.research.google.com/github/Agtiger/transformers/blob/main/StrokeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
stable = True # Set to True for latest pip version or False for main branch in GitHub
!pip install tsai -t /content/drive/MyDrive/TSAI_NEW
!pip install h5py -t /content/drive/MyDrive/TSAI_NEW
!pip install -U aeon -t /content/drive/MyDrive/AEON
import sys
print('\n'.join(sys.path))
sys.path.append('/content/drive/MyDrive/AEON')
from aeon.datasets import load_arrow_head, load_basic_motions
from aeon.transformations.collection.feature_based import TSFreshFeatureExtractor


In [ ]:
stable = True # Set to True for latest pip version or False for main branch in GitHub
!pip install tsai -t /TSAI_NEW
!pip install h5py -t /TSAI_NEW

In [ ]:
import sys
print('\n'.join(sys.path))
sys.path.append('/content/drive/MyDrive/TSAI_NEW')

In [ ]:
import numpy as np
import scipy.io
from google.colab import drive

# Mount Google Drive if the file is stored there
#drive.mount('/content/drive')

# Specify the file path
file_path = '/content/drive/MyDrive/TSAI_NEW/data/Data_Marker.mat'  # Update this path
file_path2 = '/content/drive/MyDrive/TSAI_NEW/data/target.mat'  # Update this path
# Load the .mat file
try:
    mat_data = scipy.io.loadmat(file_path)
    mat_data2 = scipy.io.loadmat(file_path2)
    print("Keys in the .mat file:", mat_data.keys())  # Show available keys
    print("Keys in the .mat file:", mat_data2.keys())  # Show available keys
    # Access the variable (update 'variable1' to your actual variable name)
    variable_name = 'Data_Marker'  # Change this to your actual variable name
    variable_name2 = 'target'  # Change this to your actual variable name
    if variable_name in mat_data and variable_name2 in mat_data2:
        data = mat_data[variable_name]
        data2 = mat_data2[variable_name2]
        print("Data shape:", data.shape)
        print("Data2 shape:", data2.shape)
    else:
        print(f"Variable '{variable_name}' not found in the file.")
except Exception as e:
    print("An error occurred:", e)




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tsai.all import *
from IPython.display import clear_output
from sklearn.metrics import *
import sklearn.metrics as skm
from fastai.metrics import *
from fastai.callback.all import EarlyStoppingCallback
from fastai.vision.all import *
from fastai.callback.tracker import EarlyStoppingCallback
my_setup()

In [ ]:
X=data
y=np.squeeze(data2)
#splits = get_splits(y, valid_size=.2, test_size=0, random_state=23, stratify=True, shuffle=True)

splits = get_splits(y, n_splits=5, shuffle=True, stratify=True)
for s in splits:
    print(np.mean(y[s[0]]), np.mean(y[s[1]]))
splits

In [ ]:
class MySuperCustomModel4(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=32, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.5, bidirectional=True, shuffle=False,
                 fc_dropout=0.,  nf=32, nb_filters=None, adaptive_size=50, **kwargs):
        super(MySuperCustomModel4,self).__init__()
        #self.lstm = nn.LSTM(input_size=c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'
        self.rnn = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias, batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0,2,1) if not shuffle else noop
        self.lstm = nn.LSTM(c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        nf = ifnone(nf, nb_filters)
        self.block = XceptionBlock(hidden_size*2, nf, **kwargs)
        self.head_nf = nf * 32
        self.head = nn.Sequential(nn.AdaptiveAvgPool1d(adaptive_size),
                                  ConvBlock(self.head_nf, self.head_nf//2, 1),
                                  ConvBlock(self.head_nf//2, self.head_nf//4, 1),
                                  ConvBlock(self.head_nf//4, c_out, 1),
                                  GAP1d(1))
    def forward(self, x):
        rnn_input = self.shuffle(x) # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        output, _ = self.rnn(rnn_input)
        last_out = output[:, -1] # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)
        output2=self.shuffle(output)
        x = self.block(output2)
        x = self.head(x)
        return x

In [ ]:
class MySuperCustomModel5(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=True, shuffle=False,
                 fc_dropout=0.,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):
        super(MySuperCustomModel5,self).__init__()
        #self.lstm = nn.LSTM(input_size=c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'
        self.LSTM = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias, batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0,2,1) if not shuffle else noop
        #self.lstm = nn.LSTM(c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        #Xception
        nf = ifnone(nf, nb_filters)
        self.block = XceptionBlock(c_in, nf, **kwargs)
        self.head_nf = nf * 32
        self.head = nn.Sequential(nn.AdaptiveAvgPool1d(adaptive_size),
                                  ConvBlock(self.head_nf, self.head_nf//2, 1),
                                  ConvBlock(self.head_nf//2, self.head_nf//4, 1),
                                  ConvBlock(self.head_nf//4, c_out, 1),
                                  GAP1d(1))
        # Common
        self.concat = Concat()
        self.fc_dropout = nn.Dropout(fc_dropout) if fc_dropout else noop
        self.fc = nn.Linear(hidden_size * (1 + bidirectional) + c_out, c_out)

    def forward(self, x):
        # xCEPTION
        x1 = self.block(x)
        y = self.head(x1)
        #print(x.shape)

        # LSTM
        rnn_input = self.shuffle(x) # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        #print(rnn_input.shape)
        output, _ = self.LSTM(rnn_input)
        last_out = output[:, -1] # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)
        #print(last_out.shape)

        # Concat
        x_con = self.concat([last_out, y])
        x_con = self.fc_dropout(x_con)
        x_con = self.fc(x_con)

        return x_con

In [ ]:
splits = get_splits(y, n_splits=5, shuffle=True, stratify=True)

archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [(XceptionTime, {})]
arch_name='LSTTM_Xception'
results = pd.DataFrame(columns=['kk', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  model = MySuperCustomModel4
  learn = ts_learner(dls, model, metrics=accuracy)
  start = time.time()
  cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=12), ShowGraph()]
  #monitor='error_rate', min_delta=1e-3, patience=5
  learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
  kk=kk+1
  learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch_name, learner_fname='learner_'+str(kk)+arch_name)
  elapsed = time.time() - start
  vals = learn.recorder.values[-1]
  dls2 = learn.dls
  valid_dl = dls2.valid
  #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
  #test_dl = valid_dl.new(valid_dl)
  test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
  accurary=(test_targets == test_preds).float().mean()
  ac=skm.accuracy_score(test_targets, test_preds)
  pre=skm.precision_score(test_targets, test_preds, average='weighted')
  f1=skm.f1_score(test_targets, test_preds, average='weighted')
  auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
  recal=skm.recall_score(test_targets, test_preds, average='weighted')
  results.loc[kk] = [kk, vals[0], vals[1], ac,pre,f1,auc,recal, int(elapsed)]
  results.sort_values(by='kk', ascending=True, kind='stable', ignore_index=True, inplace=True)
  clear_output()
  display(results)

In [ ]:
splits = get_splits(y, n_splits=5, shuffle=True, stratify=True)

archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [(XceptionTime, {})]
arch_name='LSTTM_Xception_contact'
results = pd.DataFrame(columns=['kk', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  model = MySuperCustomModel5
  learn = ts_learner(dls, model, metrics=accuracy)
  start = time.time()
  cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=15), ShowGraph()]
  #monitor='error_rate', min_delta=1e-3, patience=5
  learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
  kk=kk+1
  learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch_name, learner_fname='learner_'+str(kk)+arch_name)
  elapsed = time.time() - start
  vals = learn.recorder.values[-1]
  dls2 = learn.dls
  valid_dl = dls2.valid
  #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
  #test_dl = valid_dl.new(valid_dl)
  test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
  accurary=(test_targets == test_preds).float().mean()
  ac=skm.accuracy_score(test_targets, test_preds)
  pre=skm.precision_score(test_targets, test_preds, average='weighted')
  f1=skm.f1_score(test_targets, test_preds, average='weighted')
  auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
  recal=skm.recall_score(test_targets, test_preds, average='weighted')
  results.loc[kk] = [kk, vals[0], vals[1], ac,pre,f1,auc,recal, int(elapsed)]
  results.sort_values(by='kk', ascending=True, kind='stable', ignore_index=True, inplace=True)
  clear_output()
  display(results)

In [ ]:
splits = get_splits(y, n_splits=5, shuffle=True, stratify=True)

archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [ (FCN, {})]

results = pd.DataFrame(columns=['kk','arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  #dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize()])
  for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = ts_learner(dls, model, metrics=accuracy)
    start = time.time()
    cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=20), ShowGraph()]
    #monitor='error_rate', min_delta=1e-3, patience=5
    learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
    kk=kk+1
    learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch.__name__, learner_fname='learner_'+str(kk)+arch.__name__)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]

    dls2 = learn.dls
    valid_dl = dls2.valid
    #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
    #test_dl = valid_dl.new(valid_dl)
    test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
    accurary=(test_targets == test_preds).float().mean()
    ac=skm.accuracy_score(test_targets, test_preds)
    pre=skm.precision_score(test_targets, test_preds, average='weighted')
    f1=skm.f1_score(test_targets, test_preds, average='weighted')
    auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
    recal=skm.recall_score(test_targets, test_preds, average='weighted')
    results.loc[kk] = [kk,arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2],pre,f1,auc,recal, int(elapsed)]
    results.sort_values(by='arch', ascending=True, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)

In [ ]:
#splits = get_splits(y, n_splits=5, shuffle=True, stratify=True)

archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [ (ResCNN, {})]

results = pd.DataFrame(columns=['kk','arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  #dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize()])
  for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = ts_learner(dls, model, metrics=accuracy)
    start = time.time()
    cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=30), ShowGraph()]
    #monitor='error_rate', min_delta=1e-3, patience=5
    learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
    kk=kk+1
    learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='32_16_3'+str(kk)+arch.__name__, learner_fname='learner_32_16_3_'+str(kk)+arch.__name__)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]

    dls2 = learn.dls
    valid_dl = dls2.valid
    #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
    #test_dl = valid_dl.new(valid_dl)
    interp = ClassificationInterpretation.from_learner(learn)
    interp.plot_confusion_matrix()
    test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
    accurary=(test_targets == test_preds).float().mean()
    ac=skm.accuracy_score(test_targets, test_preds)
    pre=skm.precision_score(test_targets, test_preds, average='weighted')
    f1=skm.f1_score(test_targets, test_preds, average='weighted')
    auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
    recal=skm.recall_score(test_targets, test_preds, average='weighted')
    results.loc[kk] = [kk,arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2],pre,f1,auc,recal, int(elapsed)]
    results.sort_values(by='arch', ascending=True, kind='stable', ignore_index=True, inplace=True)
    #clear_output()
    display(results)

In [ ]:
# @title accuracy

from matplotlib import pyplot as plt
results['accuracy'].plot(kind='line', figsize=(8, 4), title='accuracy')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
bs = 64
print(X.shape)
tfms  = [None, [Categorize()]]

dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])

archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    #metrics = [accuracy, precision_multi, recall_multi,  specificity_multi, F1_multi]
    learn = ts_learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_'+arch.__name__, learner_fname='learner_'+arch.__name__)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)


In [ ]:
#learn.save_all(path='export', dls_fname='dls', model_fname='model_'+arch.__name__, learner_fname='learner_'+arch.__name__)
#del learn, dsets, dls
rchs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss','accuracy','precision','f1','auc','recall'])
for i, (arch, k) in enumerate(archs):
  learn = load_learner_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_'+arch.__name__, learner_fname='learner_'+arch.__name__)
  dls2 = learn.dls
  valid_dl = dls2.valid
  #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
  #test_dl = valid_dl.new(valid_dl)
  test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
  accurary=(test_targets == test_preds).float().mean()
  ac=skm.accuracy_score(test_targets, test_preds)
  pre=skm.precision_score(test_targets, test_preds, average='weighted')
  f1=skm.f1_score(test_targets, test_preds, average='weighted')
  auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
  recal=skm.recall_score(test_targets, test_preds, average='weighted')
  ac,pre,f1,auc,recal
  model = create_model(arch, dls=dls, **k)
  #interp = ClassificationInterpretation.from_learner(learn)
  #interp.plot_confusion_matrix()
  vals = learn.recorder.values[-1]
  results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], ac,pre,f1,auc,recal]
  results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
  clear_output()
  display(results)
accurary,ac,pre,f1,auc,recal

In [ ]:
#*****************************************backup

bs = 64
print(X.shape)
tfms  = [None, [Categorize()]]

dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])

archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
#archs = [(LSTM_FCN, {})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    metrics = [accuracy, precision_multi, recall_multi,  specificity_multi, F1_multi]
    learn = ts_learner(dls, model,  metrics=accuracy)

    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    learn.save_all(path='export', dls_fname='dls', model_fname='model_'+arch.__name__, learner_fname='learner_'+arch.__name__)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2],pre,f1,auc,recal,int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)


In [ ]:
bs = 64
print(X.shape)
tfms  = [None, [Categorize()]]

dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])

archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [(LSTM_Inception, {})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    metrics = [accuracy, precision_multi, recall_multi,  specificity_multi, F1_multi]
    learn = ts_learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2],pre,f1,auc,recal,int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, kind='stable', ignore_index=True, inplace=True)
    clear_output()
    display(results)


In [ ]:
# modified
class MySuperCustomModel2(nn.Sequential):
    def __init__(self, c_in, c_out,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):
        backbone = c_in
        head = MySuperCustomModelHead(c_in, c_out, nf, nb_filters, adaptive_size, **kwargs)
        super().__init__(OrderedDict([]))

class MySuperCustomModel1(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.):
        backbone = MySuperCustomLSTM(c_in, c_out, seq_len, hidden_size, rnn_layers, bias, cell_dropout, rnn_dropout, bidirectional, shuffle,fc_dropout)
        #head = MySuperCustomModelHead(c_in, c_out, nf, nb_filters, adaptive_size, **kwargs)
        super().__init__(OrderedDict([('backbone', backbone)]))

class MySuperCustomModel3(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):
        backbone = MySuperCustomLSTM(c_in, c_out, seq_len, hidden_size, rnn_layers, bias, cell_dropout, rnn_dropout, bidirectional, shuffle,fc_dropout)
        head = MySuperCustomModelHead(c_out, nf, nb_filters, adaptive_size, **kwargs)
        super().__init__(OrderedDict([('backbone', backbone),('head', head)]))

class MySuperCustomModel4(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):
        super(MySuperCustomModel4,self).__init__()
        #self.lstm = nn.LSTM(input_size=c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'
        self.rnn = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias, batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0,2,1) if not shuffle else noop
        self.lstm = nn.LSTM(c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        nf = ifnone(nf, nb_filters)
        self.block = XceptionBlock(c_in, nf, **kwargs)
        self.head_nf = nf * 32
        self.head = nn.Sequential(nn.AdaptiveAvgPool1d(adaptive_size),
                                  ConvBlock(self.head_nf, self.head_nf//2, 1),
                                  ConvBlock(self.head_nf//2, self.head_nf//4, 1),
                                  ConvBlock(self.head_nf//4, c_out, 1),
                                  GAP1d(1))
    def forward(self, x):
        rnn_input = self.shuffle(x) # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        output, _ = self.rnn(rnn_input)
        last_out = output[:, -1] # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)

        x = self.block(output)
        x = self.head(x)
        return x





class MySuperCustomModel2(nn.Sequential):

    def __init__(self, c_in, c_out,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):

        backbone = XceptionBlock(c_in, nf, **kwargs)
        head = MySuperCustomModelHead(nf, c_out)
        super().__init__(OrderedDict([('backbone', backbone), ('head', head)]))


class MySuperCustomModelHead(Module):

    def __init__(self, nf, c_out, adaptive_size=50, **kwargs):
        self.head_nf = nf * 32
        self.head = nn.Sequential(nn.AdaptiveAvgPool1d(adaptive_size),
                                  ConvBlock(self.head_nf, self.head_nf//2, 1),
                                  ConvBlock(self.head_nf//2, self.head_nf//4, 1),
                                  ConvBlock(self.head_nf//4, c_out, 1),
                                  GAP1d(1))

    def forward(self, x):
        x = self.head(x)
        return x.clamp(max=1.0)

class MySuperCustomLSTM(Module):

    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.):
        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'
        self.rnn = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias, batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0,2,1) if not shuffle else noop
        self.lstm = nn.LSTM(c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)
    def forward(self, x):
        rnn_input = self.shuffle(x) # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        output, _ = self.rnn(rnn_input)
        last_out = output[:, -1] # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)
        #out, _ = self.lstm(x)
        #out = out.permute(0, 2, 1)
        print(output.shape)
        return last_out

In [ ]:
# modified
class MySuperCustomModel2(nn.Sequential):
    def __init__(self, c_in, c_out,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):
        backbone = c_in
        head = MySuperCustomModelHead(c_in, c_out, nf, nb_filters, adaptive_size, **kwargs)
        super().__init__(OrderedDict([]))

class MySuperCustomModel1(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.):
        backbone = MySuperCustomLSTM(c_in, c_out, seq_len, hidden_size, rnn_layers, bias, cell_dropout, rnn_dropout, bidirectional, shuffle,fc_dropout)
        #head = MySuperCustomModelHead(c_in, c_out, nf, nb_filters, adaptive_size, **kwargs)
        super().__init__(OrderedDict([('backbone', backbone)]))

class MySuperCustomModel3(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):
        backbone = MySuperCustomLSTM(c_in, c_out, seq_len, hidden_size, rnn_layers, bias, cell_dropout, rnn_dropout, bidirectional, shuffle,fc_dropout)
        head = MySuperCustomModelHead(c_out, nf, nb_filters, adaptive_size, **kwargs)
        super().__init__(OrderedDict([('backbone', backbone),('head', head)]))

class MySuperCustomModel4(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.,  nf=64, nb_filters=None, adaptive_size=50, **kwargs):
        super(MySuperCustomModel4,self).__init__()
        #self.lstm = nn.LSTM(input_size=c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'
        self.rnn = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias, batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0,2,1) if not shuffle else noop
        self.lstm = nn.LSTM(c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        nf = ifnone(nf, nb_filters)
        self.block = XceptionBlock(hidden_size, nf, **kwargs)
        self.head_nf = nf * 32
        self.head = nn.Sequential(nn.AdaptiveAvgPool1d(adaptive_size),
                                  ConvBlock(self.head_nf, self.head_nf//2, 1),
                                  ConvBlock(self.head_nf//2, self.head_nf//4, 1),
                                  ConvBlock(self.head_nf//4, c_out, 1),
                                  GAP1d(1))
    def forward(self, x):
        rnn_input = self.shuffle(x) # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        output, _ = self.rnn(rnn_input)
        last_out = output[:, -1] # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)
        output=self.shuffle(output)
        x = self.block(output)
        x = self.head(x)
        return x





class MySuperCustomModel2(nn.Sequential):

    def __init__(self, c_in, c_out,  nf=16, nb_filters=None, adaptive_size=50, **kwargs):

        backbone = XceptionBlock(c_in, nf, **kwargs)
        head = MySuperCustomModelHead(nf, c_out)
        super().__init__(OrderedDict([('backbone', backbone), ('head', head)]))


class MySuperCustomModelHead(Module):

    def __init__(self, nf, c_out, adaptive_size=50, **kwargs):
        self.head_nf = nf * 32
        self.head = nn.Sequential(nn.AdaptiveAvgPool1d(adaptive_size),
                                  ConvBlock(self.head_nf, self.head_nf//2, 1),
                                  ConvBlock(self.head_nf//2, self.head_nf//4, 1),
                                  ConvBlock(self.head_nf//4, c_out, 1),
                                  GAP1d(1))

    def forward(self, x):
        x = self.head(x)
        return x.clamp(max=1.0)

class MySuperCustomLSTM(Module):

    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=False, shuffle=False,
                 fc_dropout=0.):
        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'
        self.rnn = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias, batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0,2,1) if not shuffle else noop
        self.lstm = nn.LSTM(c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)
    def forward(self, x):
        rnn_input = self.shuffle(x) # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        output, _ = self.rnn(rnn_input)
        last_out = output[:, -1] # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)
        #out, _ = self.lstm(x)
        #out = out.permute(0, 2, 1)
        print(output.shape)
        return last_out

In [ ]:
class LSTM_Inception(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, rnn_layers=1, bias=True, cell_dropout=0, rnn_dropout=0.8, bidirectional=True, shuffle=False,
                 fc_dropout=0.,  nf=32, nb_filters=None,  **kwargs):
        super(LSTM_Inception,self).__init__()
        #self.lstm = nn.LSTM(input_size=c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        if shuffle: assert seq_len is not None, 'need seq_len if shuffle=True'
        self.LSTM = nn.LSTM(seq_len if shuffle else c_in, hidden_size, num_layers=rnn_layers, bias=bias, batch_first=True,
                              dropout=cell_dropout, bidirectional=bidirectional)
        self.rnn_dropout = nn.Dropout(rnn_dropout) if rnn_dropout else noop
        self.shuffle = Permute(0,2,1) if not shuffle else noop
        #self.lstm = nn.LSTM(c_in, hidden_size=hidden_size, num_layers=rnn_layers, batch_first=True)

        #Inception
        nf = ifnone(nf, nb_filters) # for compatibility
        self.inceptionblock = InceptionBlock(c_in, nf, **kwargs)
        self.gap = GAP1d(1)
        self.fc = nn.Linear(nf * 4, c_out)

        # Common
        self.concat = Concat()
        self.fc_dropout = nn.Dropout(fc_dropout) if fc_dropout else noop
        self.fc2 = nn.Linear(hidden_size * (1 + bidirectional) + c_out, c_out)

    def forward(self, x):
        # xCEPTION
        x1 = self.inceptionblock(x)
        x2 = self.gap(x1)
        y = self.fc(x2)
        #print(x.shape)

        # LSTM
        rnn_input = self.shuffle(x) # permute --> (batch_size, seq_len, n_vars) when batch_first=True
        #print(rnn_input.shape)
        output, _ = self.LSTM(rnn_input)
        last_out = output[:, -1] # output of last sequence step (many-to-one)
        last_out = self.rnn_dropout(last_out)
        #print(last_out.shape)

        # Concat
        x_con = self.concat([last_out, y])
        x_con = self.fc_dropout(x_con)
        x_con = self.fc2(x_con)

        return x_con

In [ ]:
splits = get_splits(y, n_splits=5, shuffle=True, stratify=True)
#**************************************************************************************************************


In [ ]:
archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [(XceptionTime, {})]
arch_name='LSTM_Inception_dep3_ks8_nf32'
depth=3
ks=8
nf=32
results = pd.DataFrame(columns=['kk', 'train loss', 'valid loss', 'actrain','accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  c_in = 33  # Number of input features
  c_out = 5  # Number of output features
  model = LSTM_Inception(c_in=c_in, c_out=c_out, nf=nf,depth=depth,ks=ks)
  #model = LSTM_Inception(nf=17)
  learn = ts_learner(dls, model, metrics=accuracy)
  learn.recorder.train_metrics = True
  learn.recorder.valid_metrics = True
  start = time.time()
  cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=30), ShowGraph()]
  #monitor='error_rate', min_delta=1e-3, patience=5
  learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
  kk=kk+1
  #learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch_name, learner_fname='learner_'+str(kk)+arch_name)
  elapsed = time.time() - start
  vals = learn.recorder.values[-1]
  dls2 = learn.dls
  valid_dl = dls2.valid
  train_dl = dls2.train
  #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
  #test_dl = valid_dl.new(valid_dl)
  interp = ClassificationInterpretation.from_learner(learn)
  interp.plot_confusion_matrix()
  test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
  train_probas, train_targets, train_preds=learn.get_preds(dl=train_dl, with_decoded=True, save_preds=None, save_targs=None)
  actrain=skm.accuracy_score(train_targets, train_preds)
  accurary=(test_targets == test_preds).float().mean()
  ac=skm.accuracy_score(test_targets, test_preds)
  pre=skm.precision_score(test_targets, test_preds, average='weighted')
  f1=skm.f1_score(test_targets, test_preds, average='weighted')
  auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
  recal=skm.recall_score(test_targets, test_preds, average='weighted')
  results.loc[kk] = [kk, vals[0], vals[1],actrain, ac,pre,f1,auc,recal, int(elapsed)]
  results.sort_values(by='kk', ascending=True, kind='stable', ignore_index=True, inplace=True)
  #clear_output()
  display(results)
  accurary,ac,pre,f1,auc,recal

In [ ]:
arch_name='LSTTM_Inception_contact'
kk=3
learn = load_learner_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch_name, learner_fname='learner_'+str(kk)+arch_name)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()


In [ ]:
archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [(XceptionTime, {})]
arch_name='LSTM_Inception_dep3_ks16_nf8'
depth=3
ks=16
nf=8
results = pd.DataFrame(columns=['kk', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  c_in = 33  # Number of input features
  c_out = 5  # Number of output features
  model = LSTM_Inception(c_in=c_in, c_out=c_out, nf=nf,depth=depth,ks=ks)
  #model = LSTM_Inception(nf=17)
  learn = ts_learner(dls, model, metrics=accuracy)
  start = time.time()
  cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=30), ShowGraph()]
  #monitor='error_rate', min_delta=1e-3, patience=5
  learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
  kk=kk+1
  learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch_name, learner_fname='learner_'+str(kk)+arch_name)
  elapsed = time.time() - start
  vals = learn.recorder.values[-1]
  dls2 = learn.dls
  valid_dl = dls2.valid
  #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
  #test_dl = valid_dl.new(valid_dl)
  interp = ClassificationInterpretation.from_learner(learn)
  interp.plot_confusion_matrix()
  test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
  accurary=(test_targets == test_preds).float().mean()
  ac=skm.accuracy_score(test_targets, test_preds)
  pre=skm.precision_score(test_targets, test_preds, average='weighted')
  f1=skm.f1_score(test_targets, test_preds, average='weighted')
  auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
  recal=skm.recall_score(test_targets, test_preds, average='weighted')
  results.loc[kk] = [kk, vals[0], vals[1], ac,pre,f1,auc,recal, int(elapsed)]
  results.sort_values(by='kk', ascending=True, kind='stable', ignore_index=True, inplace=True)
  #clear_output()
  display(results)
  accurary,ac,pre,f1,auc,recal

In [ ]:
archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [(XceptionTime, {})]
arch_name='LSTM_Inception_dep3_ks16_nf16'
depth=3
ks=16
nf=16
results = pd.DataFrame(columns=['kk', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  c_in = 33  # Number of input features
  c_out = 5  # Number of output features
  model = LSTM_Inception(c_in=c_in, c_out=c_out, nf=nf,depth=depth,ks=ks)
  #model = LSTM_Inception(nf=17)
  learn = ts_learner(dls, model, metrics=accuracy)
  start = time.time()
  cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=70), ShowGraph()]
  #monitor='error_rate', min_delta=1e-3, patience=5
  learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
  kk=kk+1
  learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch_name, learner_fname='learner_'+str(kk)+arch_name)
  elapsed = time.time() - start
  vals = learn.recorder.values[-1]
  dls2 = learn.dls
  valid_dl = dls2.valid
  #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
  #test_dl = valid_dl.new(valid_dl)
  interp = ClassificationInterpretation.from_learner(learn)
  interp.plot_confusion_matrix()
  test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
  accurary=(test_targets == test_preds).float().mean()
  ac=skm.accuracy_score(test_targets, test_preds)
  pre=skm.precision_score(test_targets, test_preds, average='weighted')
  f1=skm.f1_score(test_targets, test_preds, average='weighted')
  auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
  recal=skm.recall_score(test_targets, test_preds, average='weighted')
  results.loc[kk] = [kk, vals[0], vals[1], ac,pre,f1,auc,recal, int(elapsed)]
  results.sort_values(by='kk', ascending=True, kind='stable', ignore_index=True, inplace=True)
  #clear_output()
  display(results)
  accurary,ac,pre,f1,auc,recal

In [ ]:
archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}),
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}),
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True})]

archs = [(LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]
archs = [(XceptionTime, {})]
arch_name='LSTM_Inception_dep3_ks16_nf64'
depth=3
ks=16
nf=64
results = pd.DataFrame(columns=['kk', 'train loss', 'valid loss', 'accuracy','precision','f1','auc','recall','time'])
kk=0
for s in splits:
  bs = 32
  print(X.shape)
  tfms  = [None, [Categorize()]]
  dsets = TSDatasets(X, y, tfms=tfms, splits=s)
  dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2],batch_tfms=[TSNormalize(range=(-1, 1))])
  c_in = 33  # Number of input features
  c_out = 5  # Number of output features
  model = LSTM_Inception(c_in=c_in, c_out=c_out, nf=nf,depth=depth,ks=ks)
  #model = LSTM_Inception(nf=17)
  learn = ts_learner(dls, model, metrics=accuracy)
  start = time.time()
  cbs = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001,patience=60), ShowGraph()]
  #monitor='error_rate', min_delta=1e-3, patience=5
  learn.fit_one_cycle(100, lr_max=1e-3, cbs=cbs)
  kk=kk+1
  learn.save_all(path='/content/drive/MyDrive/TSAI_NEWexport', dls_fname='dls', model_fname='model_Fal'+str(kk)+arch_name, learner_fname='learner_'+str(kk)+arch_name)
  elapsed = time.time() - start
  vals = learn.recorder.values[-1]
  dls2 = learn.dls
  valid_dl = dls2.valid
  #test_ds = valid_dl.dataset.add_test(X[splits[1]], y[splits[1]])# In this case I'll use X and y, but this would be your test data
  #test_dl = valid_dl.new(valid_dl)
  interp = ClassificationInterpretation.from_learner(learn)
  interp.plot_confusion_matrix()
  test_probas, test_targets, test_preds = learn.get_preds(dl=valid_dl, with_decoded=True, save_preds=None, save_targs=None)
  accurary=(test_targets == test_preds).float().mean()
  ac=skm.accuracy_score(test_targets, test_preds)
  pre=skm.precision_score(test_targets, test_preds, average='weighted')
  f1=skm.f1_score(test_targets, test_preds, average='weighted')
  auc=skm.roc_auc_score(test_targets,test_probas,average='weighted',multi_class='ovo')
  recal=skm.recall_score(test_targets, test_preds, average='weighted')
  results.loc[kk] = [kk, vals[0], vals[1], ac,pre,f1,auc,recal, int(elapsed)]
  results.sort_values(by='kk', ascending=True, kind='stable', ignore_index=True, inplace=True)
  #clear_output()
  display(results)
  accurary,ac,pre,f1,auc,recal